In [ ]:
import os
import pickle
import numpy as np
import xml.etree.ElementTree as etree
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class Box:
    def __init__(self):
        self.x, self.y = float(), float()
        self.w, self.h = float(), float()
        self.c = float()
        self.prob = float()
        
def draw_boxes(boxes, im, crop_dim, box_color=(255, 0, 0)):
    PIL_image = Image.fromarray(im.copy())
    draw = ImageDraw.Draw(PIL_image)
    [xmin, xmax] = crop_dim[0]
    [ymin, ymax] = crop_dim[1]
    
    height, width, _ = im.shape
    for b in boxes:
        w = xmax - xmin
        h = ymax - ymin

        left = int((b.x - b.w / 2.) * w) + xmin
        right = int((b.x + b.w / 2.) * w) + xmin
        top = int((b.y - b.h / 2.) * h) + ymin
        bot = int((b.y + b.h / 2.) * h) + ymin

        if left < 0:
            left = 0
        if right > width - 1:
            right = width - 1
        if top < 0:
            top = 0
        if bot > height - 1: 
            bot = height - 1
        
        thickness = 3
        draw.line([(left, bot), (left, top)], fill=box_color, width=thickness)
        draw.line([(left, top), (right, top)], fill=box_color, width=thickness)
        draw.line([(right, top), (right, bot)], fill=box_color, width=thickness)
        draw.line([(right, bot), (left, bot)], fill=box_color, width=thickness)

    del draw
    return np.array(PIL_image)

In [ ]:
annot_dict = {}
img_width = 780.0
img_height = 350.0

directories = ["../dataset/1_annot", "../dataset/2_annot", "../dataset/3_annot",
               "../dataset/4_annot", "../dataset/5_annot", "../dataset/6_annot"]

In [ ]:
for read_dir in directories:
    if not os.path.exists(read_dir):
        continue
    
    print(read_dir)
    annot_dict[read_dir] = {}

    annot_files = []
    for file in os.listdir(read_dir):
        if file.endswith(".xml") and file.startswith("image"):
            annot_files.append(file)

    for annot_file in annot_files:
        tree = etree.parse(os.path.join(read_dir, annot_file))
        root = tree.getroot()
        img_name = ""
        boxes = []

        for child in root:
            if child.tag == "filename":
                img_name = child.text
            if child.tag == "object":
                bnd_box = child[-1]
                xmin = int(bnd_box[0].text)
                ymin = int(bnd_box[1].text)
                xmax = int(bnd_box[2].text)
                ymax = int(bnd_box[3].text)

                b = Box()
                b.w = float(xmax - xmin) / img_width
                b.h = float(ymax - ymin) / img_height
                b.x = (float(xmin) / img_width) + (b.w / 2)
                b.y = (float(ymin) / img_height) + (b.h / 2)
                boxes.append(b)

        annot_dict[read_dir][img_name] = boxes

In [ ]:
for k in annot_dict:
    print(k)
    print()
    
    for img in annot_dict[k]:
        print(img)
        for b in annot_dict[k][img]:
            print(b)
            print(b.x)
            print(b.y)
            print(b.w)
            print(b.h)
        print()

In [ ]:
# pickle annotation dictionary
write_file = "../dataset/annot_dict.p"
pickle.dump(annot_dict, open(write_file, "wb"))

In [ ]:
# visualize an example annotation
image = Image.open("../dataset/6/image_003.png")
image = image.convert("RGB")
image = np.array(image)
final = draw_boxes(annot_dict["../dataset/6_annot"]["image_003.png"], image, 
                   ((0, image.shape[1]), (0, image.shape[0])), (0, 255, 0))

In [ ]:
plt.imshow(final)
plt.axis("off")
plt.show()

## Recombine annotated images into video

In [ ]:
read_dir = "../dataset/2_annot"
write_dir = "../dataset/2_annot_img"

if not os.path.exists(write_dir):
    os.makedirs(write_dir)

for k in annot_dict[read_dir]:
    image = Image.open(os.path.join("../dataset/2", k))
    image = image.convert("RGB")
    image = np.array(image)
    final = draw_boxes(annot_dict[read_dir][k], image, 
                       ((0, image.shape[1]), (0, image.shape[0])), (0, 255, 0))
    plt.imsave(os.path.join(write_dir, k), final)

In [ ]:
def combine_video_libx264(read_dir, write_dir, write_file_name):
    command = "ffmpeg -r 30 -i " + os.path.join(read_dir, "image_%03d.png") + " -vcodec libx264 " + \
               os.path.join(write_dir, write_file_name)
    print(command)
    os.system(command)

In [ ]:
write_dir = "../dataset"
combine_video_libx264("../dataset/2_annot_img", write_dir, "2_annot.mp4")